In [95]:
library(dplyr)
library(tidyr)
library(stringr)
library(copula)
library(ggplot2)

### Ex 1: Archimedean and Elliptical Copulas

In [96]:
# time period in years
T.years <- 3
t.freq <- "daily"

In [97]:
# sector ETF, SPY data
getSymbols(c(
  "SPY",  # S&P 500
  "XLY",  # Consumer Discretionary
  "XLP",  # Consumer Staples
  "XLE",  # Energy
  "XLF",  # Financials
  "XLV",  # Healthcare
  "XLI",  # Industrials
  "XLB",  # Materials
  "XLK",  # Technology
  "XLU"   # Utilities
), from = Sys.Date() - 365 * T.years)

merged_data <- na.omit(merge.xts(
  Ad(SPY),
  Ad(XLY),
  Ad(XLP),
  Ad(XLE),
  Ad(XLF),
  Ad(XLV),
  Ad(XLI),
  Ad(XLB),
  Ad(XLK),
  Ad(XLU)
))

# column names
colnames(merged_data) <- c(
  "SP500",
  "ConsumerDisc",
  "ConsumerStaples",
  "Energy",
  "Financials",
  "Healthcare",
  "Industrials",
  "Materials",
  "Technology",
  "Utilities"
)


# get returns dataframe
returns_data <- merged_data %>%
  fortify.zoo() %>%
  pivot_longer(-Index, names_to = "Asset", values_to = "Price") %>%
  group_by(Asset) %>%
  tq_transmute(
    select = Price,
    mutate_fun = periodReturn,
    period = t.freq,
    type = "log",
    col_rename = "Return"
  ) %>%
  pivot_wider(names_from = Asset, values_from = Return)

[1] "SPY" "XLY" "XLP" "XLE" "XLF" "XLV" "XLI" "XLB" "XLK" "XLU"

In [98]:
# pairs to analyze
pairs <- list(
  returns_data[, c("SP500", "Financials")],
  returns_data[, c("SP500", "Technology")],
  returns_data[, c("SP500", "Energy")],
  returns_data[, c("Financials", "Technology")],
  returns_data[, c("Technology", "Energy")],
  returns_data[, c("Financials", "Energy")]
)

# copula models to compare
copulas <- list(
  "Clayton" = claytonCopula(),
  "Gumbel" = gumbelCopula(),
  "Frank" = frankCopula(),
  "Gaussian" = normalCopula(),
  "t" = tCopula()
)

# sample size
print(paste("N = ", nrow(returns_data)))

[1] "N =  752"


In [ ]:
n <- length(pairs)
dev.off()
png("copulas.png", width = 600 * 7, height = 600 * n + 5, res = 200)
options(repr.plot.width = 9, repr.plot.height = 21)
par(mfrow = c(n, 7), mar = c(4.5, 4.5, 2.2, 2.2), oma = c(0, 0, 4, 5))

par(
  bg = "gray97",
  col.axis = "gray30",
  col.lab = "gray20",
  col.main = "black",
  bty = "n",
  family = "sans"
)

summary <- data.frame(
  Pair = character(),
  Copula = character(),
  Parameter = numeric(),
  LogLik = numeric(),
  AIC = numeric(),
  TailProb = numeric()
)

for (i in seq_along(pairs)) {
  pair <- pairs[[i]]
  x <- do.call(c, pair[, 1])
  y <- do.call(c, pair[, 2])
  u.x <- rank(x) / (length(x) + 1)
  u.y <- rank(y) / (length(y) + 1)

  plot(x, y, title(paste("Scatter Plot:\n",
                         colnames(pair)[1], "and", colnames(pair)[2])))
  plot(u.x, u.y, title(paste("Empirical Copula:\n",
                             colnames(pair)[1], "and", colnames(pair)[2])))

  for (cop_name in names(copulas)) {
    copula <- copulas[[cop_name]]
    fitted_copula <- fitCopula(copula, cbind(u.x, u.y), method = "ml")
    if (cop_name == "t") {
      fitted_copula@copula@parameters[2] <-
        round(fitted_copula@copula@parameters[2])
        fitted_copula@estimate <- round(fitted_copula@estimate[2], 2)
    }

    u_hat <- rCopula(length(x), fitted_copula@copula)
    u_hat.x <- u_hat[, 1]
    u_hat.y <- u_hat[, 2]


    # summary
    tail_prob_5 <- round(pCopula(c(0.05, 0.05),
                                 copula = fitted_copula@copula), 5)
    tail_prob_1 <- round(pCopula(c(0.01, 0.01),
                                 copula = fitted_copula@copula), 5)
    summary <- rbind(
      summary,
      data.frame(
        Pair = paste(colnames(pair)[1], "and", colnames(pair)[2]),
        Copula = cop_name,
        Parameter = fitted_copula@estimate,
        LogLik = fitted_copula@loglik,
        AIC = AIC(fitted_copula),
        "TailProb5" = tail_prob_5,
        "TailProb1" = tail_prob_1
      )
    )

    # plot empirical and estimated copulas
    plot(u_hat.x, u_hat.y, title(paste("Simulated ", cop_name, " Copula:\n",
                                       colnames(pair)[1],
                                       "and", colnames(pair)[2])))

    mtext("Empirical and Estimated Copulas",
          side = 3, line = 0.5, cex = 1.5, font = 2, outer = TRUE)
  }
}

In [ ]:
# summary table
dev.off()
png("summary.png", width = 1400, height = 700 * n / 3, res = 150)
summary <- summary[order(-rank(summary$Pair), summary$AIC), ]
colnames(summary)[colnames(summary) == "TailProb5"] <- "TailProb (5%)"
colnames(summary)[colnames(summary) == "TailProb1"] <- "TailProb (1%)"
row.names(summary) <- NULL
grid.table(summary)
dev.off()